In [293]:
from tkinter import *
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg,
    NavigationToolbar2Tk
)
import numpy as np

<h4> Канонічне рівняння прямої </h4?

$(x_1, y_1)$, $(x_2, y_2)$ -- координати 2-х точок прямої
 
$(x, y)$ -- довільна точка на площині
 
Канонічне рівняння прямої
$$
   \frac{x - x_1}{x_2 - x_1} = \frac{y - y_1}{y_2 - y_1}
$$

$$
  f = \frac{x - x_1}{x_2 - x_1} - \frac{y - y_1}{y_2 - y_1}
$$

In [296]:
# Результат підстановки координат точки в канонічне
# рівняння прямої, що проходить через 2 точки

# line = (point1, point2) -- 2 точки, що належать прямій
# point = (x, y) -- довільна точка на площині
def f(line, point) :
    point1, point2 = line    
    x1, y1 = point1
    x2, y2 = point2
    x, y   = point
    f_value = (x-x1)*(y2-y1) - (x2-x1)*(y-y1)
    return f_value

**Допоміжні функції**

In [298]:
def polygon_vertex_to_xlist_and_ylist(polygon):    
    xlist = [vertex[0] for vertex in polygon]
    ylist = [vertex[1] for vertex in polygon]
    return (xlist, ylist)

In [299]:
def line_segment(left, right, line) :
    point1, point2 = line
    x1, y1 = point1
    x2, y2 = point2

    xmin, ymin = left
    xmax, ymax = right    
    
    if x1 == x2 : # прямая x = const
        xleft = xright = x1
        yleft, yright = ymin, ymax
    else :        
        xleft, xright = xmin, xmax
        coeff = (y2-y1) / (x2-x1)
        yleft  = y1 + coeff * (xleft  - x1)
        yright = y1 + coeff * (xright - x1)

    xlist  = (xleft, xright)
    ylist =  (yleft, yright)
    segment = (xlist, ylist)

    return segment

**Тест перетину прямої та полігону**

In [301]:
def cross2(line, polygon) :   
    n = len(polygon) - 1 # size(P) - 1
    right = left = equal = 0    
    for i in range(n) :
        fxy = f(line, polygon[i])
        if fxy == 0 :
            equal = 1
        elif fxy > 0 : 
            right = 1
        else :
            left = 1
        if left * right == 1 :
            return 1
    return equal - 1

In [302]:
def cross2_Result(line, polygon) :
    res = cross2(line, polygon)
    if res == 0 :
        text = 'Пряма дотикається до полігону'
    elif res == 1 :
        text = 'Пряма перетинає полігон'
    else :
        text = 'Пряма не перетинає полігон'
    return text

**Тест орієнтації точки відносно прямої**

In [304]:
def nf2(a, b, p) :  
    ax, ay = a
    bx, by = b
    px, py = p
    det = (px - ax) * (by - ay) - (py - ay) * (bx - ax)
    return det

In [305]:
def nf2__result(line, p) :
    a = line[0]
    b = line[1]
    result = nf2(a, b, p)
    if result == 0 :
        return f"т.{p} лежить на прямій"
    elif result > 0 :
        return f"т.{p} лежить справа від прямої"
    else :
        return f"т.{p} лежить зліва від прямої"

**Тест опуклості полігону**

In [307]:
def conv2(polygon) :
    n = len(polygon) - 1
    c = nf2( polygon[n-1], polygon[0], polygon[1] )
    for i in range(1, n) :
        s = nf2( polygon[i-1], polygon[i], polygon[i+1] )
        t = c * s
        if t == 0 :
            return -1
        elif t < 0 :
            return 0
    return 1

In [308]:
def conv2__result(polygon) :
    result = conv2(polygon)
    if result == 0 :
        return 'Полігон неопуклий'
    elif result == 1 :
        return 'Полігон опуклий'
    else :
        return """В полігона є принаймні одна вершина
    з розгорнутим кутом (180°)"""

**Тест самоперетину полігону**

In [310]:
def self_test(polygon) :
    n = len(polygon)-1 
        
    for i in range(n-2) :
        if i == 0 :
            jmin, jmax = 2, n-1
        else :
            jmin, jmax = i+2, n
        for j in range(jmin, jmax) :
            f1 = nf2(polygon[i], polygon[i+1], polygon[j])
            f2 = nf2(polygon[i], polygon[i+1], polygon[j+1])
            f3 = nf2(polygon[j], polygon[j+1], polygon[i])
            f4 = nf2(polygon[j], polygon[j+1], polygon[i+1]) 
            if f1 * f2 < 0 and f3 * f4 < 0 :
                return True
    return False

In [311]:
def self_test__result(polygon) :
    result = self_test(polygon)
    if result == True :
        return "Полігон перетинає сам себе"
    else :
        return "Полігон сам себе не перетинає"

**Опуклий тест**

In [313]:
def conv2_test( polygon, point ) :        
    n = len( polygon) - 1
    right = left = equal = 0
    for i in range(n) :        
        f = nf2(polygon[i], polygon[i+1], point)
        if  f == 0 :
            equal = 1
        elif f < 0 :
            left = 1
        else :
            right = 1
        if left * right == 1 :
            return 1
    return equal - 1

In [314]:
def conv2_test_result( polygon, point ) :
    result = conv2_test( polygon, point )
    if result == 0 :
        return f"т.{point} лежить на межі полігону"
    elif result == 1 :
        return f"т.{point} лежить поза полігоном"
    else :
        return f"т.{point} лежить всередині полігону"

**Габаритний тест**

In [316]:
def gab2_test(polygon, point) :  
    polygon_x, polygon_y = polygon_vertex_to_xlist_and_ylist(polygon)
    point_x,   point_y   = point
    
    inside  = f"""Т.({point_x}, {point_y}) лежить 
    всередині габаритного прямокутника"""
    outside = f"""  Т.({point_x}, {point_y}) 
лежить поза габаритним прямокутником"""
    
    xmin, xmax = min(polygon_x), max(polygon_x)
    ymin, ymax = min(polygon_y), max(polygon_y)
    
    if xmin <= point_x <= xmax  and ymin <= point_y <= ymax :
        return inside
    else:
        return outside

**Кутовий тест: Радіальний варіант**

In [318]:
def sign(x) :
    if x == 0 :
        return 0
    elif x > 0 :
        return 1
    else :
        return -1

In [319]:
def vectorLength(vector) :    
    vx, vy = vector
    length = (vx**2 + vy**2) ** 0.5
    return length

In [320]:
def scalarProduct(vector1, vector2) :
    vx, vy = vector1
    wx, wy = vector2
    scalar_product = vx * wx + vy * wy
    return scalar_product 

In [321]:
def ang(vector1, vector2) :
    from math import acos    
    vx, vy = vector1
    wx, wy = vector2
    # определитель
    d = vx * wy - vy * wx
    if d == 0 :
        znak = 1
    else :
        znak = sign(d)
    v_length = vectorLength(vector1)
    w_length = vectorLength(vector2)
    vw = scalarProduct(vector1, vector2)
    radian_angle = znak * acos( vw / v_length / w_length )
    return radian_angle

In [322]:
def vectorDifference(vector1, vector2) :
    vx, vy = vector1
    wx, wy = vector2
    ux, uy = vx - wx, vy - wy
    return (ux, uy)

In [323]:
def rad_test(polygon, point) :
    from math import pi
    from copy import deepcopy
    
    s = delta = 0
    n = len(polygon) - 1
    
    # i = 0
    v = vectorDifference( polygon[0], point )
    v_length = vectorLength( v )
    if v_length == 0 :
        return 0
    w = deepcopy(v)    
    
    for i in range(1, n) :
        v = vectorDifference( polygon[i], point )
        v_length = vectorLength( v )
        if v_length == 0 :
            return 0
        delta = ang(w, v)
        abs_delta = abs(delta)
        if abs_delta == pi :
            return 0
        w = deepcopy(v) 
        s += delta
    return sign(pi - abs(s))

In [324]:
def rad_test__result(polygon, point) :
    result = rad_test(polygon, point)
    if result == 0 :
        return f"{point} -- гранична точка"
    elif result == -1 :
        return f"{point} -- внутрішня точка"
    else: # result = 1
        return f"{point} -- зовнішня точка"    

**Кутовий тест: Октантний варіант**

In [326]:
def oct(vector) :
    x, y = vector
    if  x == 0  and  y == 0:      return 0
    if  0 <= y <  x:              return 1
    if  0 <  x <= y:              return 2
    if -y <  x <= 0:              return 3
    if  0 <  y <= -x:             return 4
    if  x <  y <= 0:              return 5
    if  y <= x <  0:              return 6
    if  0 <= x < -y:              return 7
    if -x <= y <  0:              return 8

In [327]:
def oct_test(polygon, point) :
    n = len(polygon)
    
    s = delta = 0
    
    # i = 0
    vv = vectorDifference( polygon[0], point )
    v = oct(vv)
    if  v == 0 :  return 0
    w = v
    
    for i in range(1, n) :        
        vv = vectorDifference( polygon[i], point )
        v = oct(vv)        
        if  v == 0 :  return 0
        
        delta = v - w        
        abs_delta = abs(delta)
        if   abs_delta > 4 :
            delta -= 8 * sign(delta)
        elif abs_delta == 4 :
            f = nf2(polygon[i-1], polygon[i], point)
            if  f == 0 :  return 0
            delta = -4 * sign(f)
            
        w = v
        s += delta
        
    return 1 - 2 * sign( abs(s) )

In [328]:
def oct_test__result(polygon, point):
    result = oct_test(polygon, point)
    if result == 0 :
        return f"{point} -- гранична точка"
    elif result == -1 :
        return f"{point} -- внутрішня точка"
    elif result ==  1 :
        return f"{point} -- зовнішня точка"
    else :
        return f"{result} -- неправильний результат"

**Променевий тест**

In [330]:
def ray2_test(polygon, point) :    
    import numpy as np
    from numpy.random import uniform
    from numpy.linalg import det, inv
    
    n = len(polygon) - 1
    while True:
        f = 1
        phi = np.random.uniform(0, 2*np.pi)
        v = [np.cos(phi), np.sin(phi)]
        for i in range(n) :
            w = vectorDifference(polygon[i], polygon[i+1])
            M = np.array([v, w])            
            detM = det(M)            
            if detM != 0 :
                invM = inv(M)
                u = vectorDifference(polygon[i], point)                
                t, tau = np.dot(u, invM)
                if t == 0 :
                    if (0 < tau <= 1) :
                        return 0
                elif t > 0 :
                    if (tau == 0) or (tau == 1) :
                        break
                    elif not ((tau < 0) or (tau > 1) ) :
                        f = -f
        return f    

In [331]:
def ray2_test__result(polygon, point) :
    result = ray2_test(polygon, point)
    if   result == 0 :
        return f"т.{point} на межі полігону"
    elif result == 1 :
        return f"т.{point} поза полігоном"
    else :
        return f"т.{point} всередині полігону"

In [332]:
def createLabel(label_master, label_font, label_text) :
    return Label(master = label_master, 
                 font   = label_font,
                 text   = label_text,                 
                 #relief = GROOVE
                )

In [333]:
# Шрифты

font1 = ["Courier", 14, "bold"]
font2 = ["Courier", 12]

label_font1  = font1 
label_font2  = font2

button_font  = font1

listbox_font = font2

In [334]:
def create_figure_poly(polygon_points):
    fig = plt.figure()  # Create a figure object
    ax = fig.add_subplot(111)  # Add a single subplot to the figure

    polygon_x = [point[0] for point in polygon_points]
    polygon_y = [point[1] for point in polygon_points]

    # Plot the polygon
    ax.plot(polygon_x + [polygon_x[0]], polygon_y + [polygon_y[0]], 
             color='blue', label='Polygon')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend()

    plt.close(fig)
    
    return fig

In [335]:
def create_figure_line_poly(line_points, polygon_points):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    polygon_x = [point[0] for point in polygon_points]
    polygon_y = [point[1] for point in polygon_points]

    ax.plot(polygon_x + [polygon_x[0]], polygon_y + [polygon_y[0]], 
             color='blue', label='Polygon')

    # Plot the line
    # Calculate the line equation: y = mx + b
    x1, y1 = line_points[0]
    x2, y2 = line_points[1]

    if x1 != x2:  # Avoid division by zero
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1

        # Determine the x-range for the plot
        x_min, x_max = ax.get_xlim()
        x_values = [x_min, x_max]
        y_values = [slope * x + intercept for x in x_values]

        # Plot the extended line
        ax.plot(x_values, y_values, color='red', label='Line', linestyle='--')
        
    else:
        # Vertical line (infinite in y direction)
        y_min, y_max = ax.get_ylim()
        ax.plot([x1, x1], [y_min, y_max], color='red', label='Line', linestyle='--')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend()

    plt.close(fig)
    
    return fig

In [336]:
def create_figure_line_point(line_points, point_coords):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    # Calculate the line equation: y = mx + b
    x1, y1 = line_points[0]
    x2, y2 = line_points[1]

    point_x, point_y = point_coords
    ax.scatter(point_x, point_y, color='green', label='Point', zorder=5)

    if x1 != x2:  # Avoid division by zero
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        # Determine the x-range for the plot
        x_min, x_max = ax.get_xlim()
        x_values = [x_min, x_max]
        y_values = [slope * x + intercept for x in x_values]
        # Plot the extended line
        ax.plot(x_values, y_values, color='red', label='Line', linestyle='--')
    else:
        # Vertical line (infinite in y direction)
        y_min, y_max = ax.get_ylim()
        ax.plot([x1, x1], [y_min, y_max], color='red', label='Line', linestyle='--')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend()

    plt.close(fig)
    
    return fig

In [337]:
def create_figure_poly_point(polygon_points, point_coords):
    fig = plt.figure()
    ax = fig.add_subplot(111) 

    polygon_x = [point[0] for point in polygon_points]
    polygon_y = [point[1] for point in polygon_points]

    ax.plot(polygon_x + [polygon_x[0]], polygon_y + [polygon_y[0]], 
             color='blue', label='Polygon')

    point_x, point_y = point_coords
    ax.scatter(point_x, point_y, color='green', label='Point', zorder=5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    ax.legend()

    plt.close(fig)

    return fig

In [338]:
def picture( input_data, type ) :
    
    if type == "linePoly":
        fig = create_figure_line_poly(input_data[0], input_data[2])
    elif type == "poly":
        fig = create_figure_poly(input_data[2])
    elif type == "linePoint":
        fig = create_figure_line_point(input_data[0], input_data[1])
    elif type == "polyPoint":
        fig = create_figure_poly_point(input_data[2], input_data[1])

    window_picture = Tk()
    window_picture.title("Графік")
    window_picture.geometry(f"{main_window_width}x{main_window_height}") 
    
    label_window_name = createLabel(window_picture, label_font1, 
                                    "Графік")
    label_window_name.pack()

    canvas = FigureCanvasTkAgg(fig, master=window_picture)
    canvas.draw()
    canvas.get_tk_widget().pack()
    
  

In [339]:
def create_input_window(algorythm_type) : 
        
    def getInputData():
        line = []
        polygon = []
        point = []

        if input_line == True:
            x1 = float( entry__x1.get() )
            y1 = float( entry__y1.get() )
            point1 = (x1, y1)

            x2 = float( entry__x2.get() )
            y2 = float( entry__y2.get() )
            point2 = (x2, y2)
            
            line = (point1, point2)         
        
        if input_point == True:
            x = float( entry__x.get() )
            y = float( entry__y.get() )
            point = (x,y)

        if input_poly == True:
            polygon = []
            for i in range(n) :
                input_x = entry_x[i].get()
                input_y = entry_y[i].get()
                if input_x == "" or input_y == "" :
                    break
                x = float(input_x)
                y = float(input_y)            
                polygon.append( (x, y) )            
            
            polygon.append( polygon[0] )

        return (line, point, polygon)

    def calculate(input_data) :      
        data = getInputData()
        label_result_text = ""

        if algorythm_type == 0:
            label_result_text = cross2_Result(data[0], data[2])
        elif algorythm_type == 1:
            label_result_text = conv2__result(data[2])
        elif algorythm_type == 2:
            label_result_text = self_test__result(data[2])
        elif algorythm_type == 3:
            label_result_text =  nf2__result(data[0], data[1])
        elif algorythm_type == 4:
            label_result_text =  conv2_test_result(data[2], data[1])
        elif algorythm_type == 5:
            label_result_text =  gab2_test(data[2], data[1])
        elif algorythm_type == 6:
            label_result_text =  rad_test__result(data[2], data[1])
        elif algorythm_type == 7:
            label_result_text =  oct_test__result(data[2], data[1])
        elif algorythm_type == 8:
            label_result_text =  ray2_test__result(data[2], data[1])
        else: 
            return 0
        
        label_result["text"] = label_result_text
    
    def plot():
        data = getInputData()

        if algorythm_type == 0:
            picture(data, "linePoly")
        elif algorythm_type in [1,2]:
            picture(data, "poly")
        elif algorythm_type == 3:
            picture(data, "linePoint")
        elif algorythm_type in [4,5,6,7,8]:
            picture(data, "polyPoint")

    window_alg_data = Tk()
    window_alg_data.title("Вихідні дані алгоритму")
    window_alg_data.geometry(f"{main_window_width}x{main_window_height}")
    
    grid_rows, grid_cols = 26, 20
    
    # Конфигурація grid 
    for c in range(grid_cols): 
        window_alg_data.columnconfigure(index = c, weight = 1)
    for r in range(grid_rows): 
        window_alg_data.rowconfigure(index = r, weight = 1)        
        
    label_0 = createLabel(window_alg_data, label_font1, 
                          "Вхідні дані алгоритму")
    label_0.grid(row = 0, column = 0, columnspan = grid_cols)
    
    label_1 = createLabel(window_alg_data, label_font1, 
                          algorithms[algorythm_type])
    label_1.grid(row = 1, column = 0, columnspan = grid_cols)
    
    label_2 = createLabel(window_alg_data, label_font1, "Введіть")
    label_2.grid(row = 2, column = 0, columnspan = grid_cols)

    input_line = False
    input_point = False
    input_poly = False

    if algorythm_type in [0,3]:
        input_line = True

    if algorythm_type != 3: 
        input_poly = True

    if algorythm_type >= 3: 
        input_point = True

    if input_line == True:
        label__line_coords = createLabel(window_alg_data, label_font1, 
                                  "координати 2-х точок прямої")
        label__line_coords.grid(row = 3, column = 0, 
                                  columnspan = grid_cols)
        # Координати 1-ї точки
        label__x1 = createLabel(window_alg_data, label_font2, "x1 = ")
        label__x1.grid(row = 4, column = 0)
        
        entry__x1 = Entry(master = window_alg_data, width = 3)
        entry__x1.grid(row = 4, column = 1)
        
        label__y1 = createLabel(window_alg_data, label_font2, "y1 = ")
        label__y1.grid(row = 4, column = 2)
        
        entry__y1 = Entry(master = window_alg_data, width = 3)
        entry__y1.grid(row = 4, column = 3)

        # Координати 2-ї точки
        label__x2 = createLabel(window_alg_data, label_font2, "x2 = ")
        label__x2.grid(row = 4, column = 4)
        
        entry__x2 = Entry(master = window_alg_data, width = 3)
        entry__x2.grid(row = 4, column = 5)
        
        label__y2 = createLabel(window_alg_data, label_font2, "y2 = ")
        label__y2.grid(row = 4, column = 6)
        
        entry__y2 = Entry(master = window_alg_data, width = 3)
        entry__y2.grid(row = 4, column = 7)

    if input_point == True: 
        label__point_coords = createLabel(window_alg_data, label_font1, 
                                  "координати точки")
        label__point_coords.grid(row = 5, column = 0, 
                                  columnspan = grid_cols)
        # Координати точки
        label__x = createLabel(window_alg_data, label_font2, "x = ")
        label__x.grid(row = 6, column = 0)
        
        entry__x = Entry(master = window_alg_data, width = 3)
        entry__x.grid(row = 6, column = 1)
        
        label__y = createLabel(window_alg_data, label_font2, "y = ")
        label__y.grid(row = 6, column = 2)
        
        entry__y = Entry(master = window_alg_data, width = 3)
        entry__y.grid(row = 6, column = 3)

    n = 10  
        
    start_row = 8

    if input_poly == True:
        label__polygon_coords = createLabel(window_alg_data, label_font1, 
                                        "координати вершин полігону")
        label__polygon_coords.grid(row = 7, column = 0, 
                                        columnspan = grid_cols)    
        label_x = [Label(master = window_alg_data, 
                        text   = "x" + str(i+1), width = 3)
                for i in range(n)]
        
        for i in range(n) :
            label_x[i].grid(row = start_row + i, column = 0)
        
        entry_x = [Entry(master = window_alg_data, width = 3)
                    for i in range(n) ]
        for i in range(n) :
            entry_x[i].grid(row = start_row + i, column = 1)
            
            
        label_y = [Label(master = window_alg_data, 
                        text   = "y" + str(i+1), width = 3)
                    for i in range(n)]
        
        for i in range(n) :
            label_y[i].grid(row = start_row + i, column = 2)

        entry_y = [Entry(master = window_alg_data, width = 3)
                    for i in range(n) ]

        for i in range(n) :
            entry_y[i].grid(row = start_row + i, column = 3)
    
    input_data = []

    button_input = Button(master = window_alg_data,
                          text = "Застосувати алгоритм",
                          command = lambda arg = input_data: calculate(arg) )
    button_input.grid(row = start_row + n, column = 0, 
                      columnspan = grid_cols)
    
    label_result = createLabel(window_alg_data, label_font1, 
                               "")
    label_result.grid(row = start_row + n + 1, column = 0, 
                      columnspan = grid_cols)    
    
    button_figure = Button(master = window_alg_data,
                text = "Намалювати графік",
                command = plot)
    
    button_figure.grid(row = start_row + n + 2, column = 0, 
                           columnspan = grid_cols)

In [340]:
def main_window__button_click() :
    ind = main_window__listbox.curselection()[0]
    create_input_window(ind)
    #window_func = "create_window__" + str(ind)
        

In [341]:
main_window_width, main_window_height = 500, 500

main_window__label_text  = "Алгоритми"
main_window__button_text = "Обрати алгоритм"

In [342]:
d = 5

In [343]:
algorithms = [
    "Тест перетину прямої та полігону",         #0 пряма + полігон
    "Тест опуклості полігону",                  #1 полігон
    "Тест самоперетину полігону",               #2 полігон
    "Тест орієнтації точки відносно прямої",    #3 пряма + точка
    "Опуклий тест",                             #4 полігон + точка
    "Габаритний тест",                          #5 полігон + точка 
    "Кутовий тест: Радіальний варіант",         #6 полігон + точка 
    "Кутовий тест: Октантний варіант",          #7 полігон + точка 
    "Променевий тест"   						#8 полігон + точка 
]

In [344]:
# Создание окна
main_window = Tk()
main_window.title("Алгоритми комп'ютерної графіки")
main_window.geometry(f"{main_window_width}x{main_window_height}")

''

In [345]:
main_window__label = createLabel(main_window, 
                                 label_font1, 
                                 main_window__label_text)
main_window__label.pack()

In [346]:
main_window__listbox_item = StringVar(value = algorithms)
main_window__listbox = Listbox(master = main_window, 
                          width  = 45,
                          height = 21,
                          listvariable = main_window__listbox_item,                  
                          font = listbox_font)
main_window__listbox.pack()

In [347]:
main_window__button = Button(master  = main_window,
                        font    = button_font,
                        text    = main_window__button_text,
                        command = main_window__button_click)
main_window__button.pack(side = BOTTOM, fill = X, padx = 3*d, pady = 3*d)

In [ ]:
main_window.mainloop()